***Исследовательская работа: Анализ и оптимизаиця алгоритмов на Python*** 

**1. Введение**

В данной работе рассматривается задача симуляции стоахастических дифференциальных уравнений.

**2. Постановка задачи**

Задача заключается в том, чтобы перенести существующий код MATLAB, который моделирует некоторые стохастические процессы с помощью метода KDE (оценки плотности ядра), в Python.

**3. Описание алгоритма на Python**

- Функция *kde*, которая выполняет оценку плотности ядра (KDE) на основе предоставленных данных. Она предоставляет основу для генерации новых данных, соответствующих исходному распределению.

In [1]:
def kde(data, probability_fcn="cdf", bandwidth=None):
    if bandwidth is None:
        bandwidth = 1.0  # Default bandwidth
    kde = KernelDensity(kernel='gaussian', bandwidth=bandwidth).fit(data[:, None])
    x = np.linspace(min(data), max(data), 1000)
    log_dens = kde.score_samples(x[:, None])
    density = np.exp(log_dens)
    if probability_fcn == "cdf":
        cdf = np.cumsum(density)
        cdf /= cdf[-1]
        return cdf, x, bandwidth
    else:
        return density, x, bandwidth


- Определение параметров:

In [ ]:
if bandwidth is None:
    bandwidth = 1.0  # Default bandwidth

Здесь мы устанавливаем значение по умолчанию для ширины ядра *bandwith*. Если в функцию не передано значение ширирины ядра, используется значение 1.0. Ширина ядра влияет на гладкость оценки: меньшие значения приводят к более 'заострённой' оценке, а большие - к более 'сглаженной'.

- Инициализация модели KDE:

In [ ]:
kde = KernelDensity(kernel='gaussian', bandwidth=bandwidth).fit(data[:, None])


Здесь создаётся объект *KernelDensity* с гауссовским ядром и заданной шириной ядра. Метод *.fit* используется для тренировки модели на данных. Данные преобразуются в столбец ('data[:, None]'), что требуется для работы с библиотекой *sklearn*.

- Вычисление логарифмической плотности и плотности:

In [ ]:
x = np.linspace(min(data), max(data), 1000)
log_dens = kde.score_samples(x[:, None])
density = np.exp(log_dens)


Создаётся массив *x* с 1000 точками, равномерно распределёнными между минимальными и максимальными значениями данных. *log_dens* содержит логарифмические значения плотности для этих точек. Экспоненцирование *log_dens* даёт *density*, непосредственную оценку плотности на этих точках.

- Возвращение CDF или плотности:

In [ ]:
if probability_fcn == "cdf":
    cdf = np.cumsum(density)
    cdf /= cdf[-1]
    return cdf, x, bandwidth
else:
    return density, x, bandwidth


Если функция вызывается с *probability_fcn="cdf"*, то вычисляется сумма последовательных результатов плотностей (*cdf*), которая нормализуется, чтобы максимальное значение было равно 1. Это превращает плотность в функцию распределения вероятностей. Если же требуется вернуть саму плотность, возвращается массив *density* вместе с соответствующими значениями *x* и используемой шириной ядра *bandwith*.

- Функция *randkde* предназначена для генерации случайных выборок из распределения, плотность которого оценивается методом ядерной оценки плотности (KDE). Основная задача этой функции - использовать полученную плотность распределения для создания новых данных, которые статистически соответствуют исходному набору данных.

In [ ]:
def randkde(data, dim):
    # Remove infinities and large values from the data
    data = np.nan_to_num(data, nan=0.0, posinf=np.finfo(np.float64).max, neginf=np.finfo(np.float64).min)
    
    # Perform KDE and get the CDF
    F, x, bw = kde(data, probability_fcn="cdf")
    
    # Ensure the CDF starts at 0 and ends at 1
    F[0] = 0
    F[-1] = 1
    
    # Create an interpolated inverse CDF function
    inv_cdf = interp1d(F, x, kind='linear', fill_value="extrapolate")
    
    # Generate random samples
    total_elements = np.prod(dim)
    uniform_random_samples = np.random.rand(total_elements)
    y = inv_cdf(uniform_random_samples).reshape(dim)
    
    return y, F, x, bw


- Обработка исключений в данных:

In [ ]:
data = np.nan_to_num(data, nan=0.0, posinf=np.finfo(np.float64).max, neginf=np.finfo(np.float64).min)


Эта строка предназанчена для замены всех *NaN* значений на 0.0 положительных бесконечностей на максимально возможное значение типа *float64* и отрицательных бесконечностей на минимально возможное значение типа *float64*. Это предовращает возможные ошибки в вычислениях, связанные с неопределёнными и экстремальными значениями.

- Выполнение KDE и получение CDF:

In [ ]:
F, x, bw = kde(data, probability_fcn="cdf")


Здесь функция *kde* вызывается с параметром *probability_fcn="cdf"*, что указывает на необходимость возвращать функцию распределения (CDF), основанную на ядерной оценке плотности данных. *F* содержит значения CDF, *x* - соответствующие значения переменной, а *bw* - используемую ширину ядра.

- Корректировка CDF:

In [ ]:
F[0] = 0
F[-1] = 1


Устанавливается точное начало и конец в CDF в 0 и 1, что гарантирует правильное распределение вероятностей.

- Интерполяция для обратной функции CDF:

In [ ]:
inv_cdf = interp1d(F, x, kind='linear', fill_value="extrapolate")


Создаётся функция *inv_cdf*, которая явялется интерполированной обратной функцией CDF. Это позволяет генерировать новые значения данных, соответствующие оригинальному распределению.

- Генерация случайных выборок:

In [ ]:
uniform_random_samples = np.random.rand(total_elements)
y = inv_cdf(uniform_random_samples).reshape(dim)


Генерируется равномерно распределённые случайные числа, которые затем преобразуются в соответствии с обратной функцией CDF для получения новых данных, распределённых согласно исходному набору. Результат приводится к нужным размерностям *dim*.

- функция *filter_infinite_values*, которая служит важной задачей предварительной обработки данных, удаляя или заменяя аномальные значения, такие как бесконечности и *NaN*.

In [ ]:
def filter_infinite_values(data):
    return np.nan_to_num(data, nan=0.0, posinf=np.finfo(np.float64).max, neginf=-np.finfo(np.float64).max)


Функция *filter_infinite_values* предназначена для очистки данных, устраняя возможные проблемы, которые могут возникнуть из-за наличия *NaN* (не числа), положительных бесконечностей (*posinf*), и отрицательных бесконечностей (*neginf*). Эти значения могут вызвать ошибки или привести к неверным результатам во время математических операций, особенно при использовании функций, которые не могут обрабатывать эти специфические случаи.

- Функция *np.nan_to_num*: 

Эта функция из NumPy преобразует *NaN* в определенные числовые значения и заменяет бесконечности на максимально допустимые числа, которые может представить тип данных *float64*.

- Функция *clamp_values* ограничивает значения в массиве data заданными минимальным (*min_value*) и максимальным (*max_value*) порогами. Это обеспечивает, что все значения в массиве находятся в заданном диапазоне.

In [ ]:
def clamp_values(data, min_value, max_value):
    return np.clip(data, min_value, max_value)


*np.clip* является функцией из библиотеки NumPy, которая "обрезает" значения массива, чтобы они находились в указанном интервале.

- Функция *safe_arctanh* обеспечивает безопасное выполнение гиперболической арктангенс функции *arctanh*, которая не определена для входных значений точно равных -1 или 1 и приводит к бесконечности при их приближении.

In [ ]:
def safe_arctanh(x):
    epsilon = 1e-10
    x = np.clip(x, -1 + epsilon, 1 - epsilon)
    return np.arctanh(x)


Значения *x* ограничиваются в диапазоне от -1 + *epsilon* до 1 - *epsilon*, где *epsilon* — очень маленькое положительное число (здесь 1e-10).

После ограничения диапазона функция *np.arctanh* вызывается безопасно, так как входные данные находятся в диапазоне, где arctanh определен и возвращает конечные значения.

- Установка параметров:

In [8]:
dv = 3
dc = 6
sigma = 0.75

N = int(1e6)
mu = 2 / sigma ** 2
n = dc - 1
m = dv - 1


- Имена файлов и потоки данных:

In [ ]:
fname_pref = f'dv={dv}dc={dc}s={sigma}'
CNfile_path = f'{fname_pref}CNhist.csv'
VNfile_path = f'{fname_pref}VNhist.csv'
CNfile = open(CNfile_path, 'w')
VNfile = open(VNfile_path, 'w')


*fname_pref*: Строка, формирующая префикс имени файла на основе параметров dv, dc и sigma.
*CNfile_path, VNfile_path*: Пути к файлам, куда будут записываться результаты. Имена файлов включают параметры для удобства идентификации.
*CNfile, VNfile*: Открытие файлов для записи. Файлы открываются в режиме записи ('w'), что означает, что если файл с таким именем уже существует, он будет перезаписан.

- Подготовка к визуализации: 

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 12))


*fig, (ax1, ax2)*: Создание фигуры с двумя подграфиками (ax1 и ax2). Размер фигуры задается как 10x12 дюймов.


- Параметры для циклов итераций: 

In [ ]:
Lmax = 30
Lplot = np.arange(1, Lmax + 1)
numLastPlots = 1


- Генерация начальных значений: 

In [ ]:
VmNn = np.random.normal(mu, np.sqrt(2 * mu), (m, N, n))


Здесь *VmNn* инициализируется как трехмерный массив, содержащий значения, сгенерированные из нормального распределения с параметрами среднего (*mu*) и стандартного отклонения (*sqrt(2 * mu)*). Размер массива задается тремя параметрами: *m, N, n*.

- Фильтрация аномальных значений:

In [ ]:
VmNn = filter_infinite_values(VmNn)
VmNn = clamp_values(VmNn, -1000, 1000)


*filter_infinite_values(VmNn)*: Применяет функцию для очистки данных от бесконечных и нечисловых значений (NaN).  
*clamp_values(VmNn, -1000, 1000)*: Ограничивает значения в VmNn диапазоном от -1000 до 1000, чтобы предотвратить чрезмерно большие или малые значения, которые могут повлиять на стабильность вычислений

- Инициализация буферных массивов:

In [ ]:
Uarx = np.zeros((numLastPlots, N))
Varx = np.zeros((numLastPlots, N))
for i in range(numLastPlots):
    Varx[i, :] = VmNn[0, :, 0]


*Uarx* и *Varx* инициализируются как массивы с нулями. Их размеры зависят от *numLastPlots* и *N*, что позволяет хранить промежуточные или конечные результаты для небольшого количества последних графиков или анализов.  
Цикл копирует первый слой данных из *VmNn* в *Varx*, что может использоваться для начальных условий или специфических вычислений в дальнейшем.

- Запуск таймера:

In [ ]:
tstart = datetime.now()
tcurr = datetime.now()

- Основной цикл обработки:

In [ ]:
for L in range(1, Lmax + 1):
    if L % 10 == 0:
        print(L, datetime.now() - tcurr, datetime.now() - tstart)
        tcurr = datetime.now()

    UmN = 2 * safe_arctanh(np.prod(np.tanh(VmNn / 2), axis=2))
    UmN = filter_infinite_values(UmN)
    UmN = clamp_values(UmN, -1000, 1000)
    UmNV = np.minimum(UmN, 1000)
    V = np.sum(UmN, axis=0) + np.random.normal(mu, np.sqrt(2 * mu), N)
    V = filter_infinite_values(V)
    V = clamp_values(V, -1000, 1000)

    for i in range(numLastPlots - 1):
        Varx[i, :] = Varx[i + 1, :]
        Uarx[i, :] = Uarx[i + 1, :]
    Varx[numLastPlots - 1, :] = V
    Uarx[numLastPlots - 1, :] = UmN[0, :]

    VmNn = randkde(V, (m, N, n))[0]
    VmNn = filter_infinite_values(VmNn)
    VmNn = clamp_values(VmNn, -1000, 1000)

    converged = np.sum(V < 0) <= 0


- Цикл итераций:

*for L in range(1, Lmax + 1)*: Итерации от 1 до *Lmax* включительно, где каждая итерация соответствует одному уровню или шагу обработки данных.

- Мониторинг выполнения:

Вывод текущего шага и времени, затраченного с начала выполнения и с предыдущего вывода, если шаг кратен 10.

- Расчёт преобразования значений: 

*UmN = 2 * safe_arctanh(np.prod(np.tanh(VmNn / 2), axis=2))*: Преобразование значений *VmNn* с использованием гиперболического тангенса, его произведения по оси и безопасного арктангенса, умноженного на 2.

- Аккумуляция и суммирование значений:

*V = np.sum(UmN, axis=0) + np.random.normal(mu, np.sqrt(2 * mu), N)*: Суммирование преобразованных значений по оси и добавление шума, сгенерированного из нормального распределения.

- Обновление массивов для хранения результатов:

Перемещение старых данных в *Varx* и *Uarx* для освобождения места под новые данные.

- Генерация новых значений:

*VmNn = randkde(V, (m, N, n))[0]*: Генерация нового набора данных на основе текущих значений *V* с использованием функции *KDE*.

- Проверка на сходимость:

*converged = np.sum(V < 0) <= 0*: Проверка условия сходимости, если количество отрицательных значений в *V* равно нулю или меньше.

In [ ]:
for i in range(converged + numLastPlots * int(not converged), numLastPlots + 1):
    l = L + i - numLastPlots
    if l < L and l in Lplot:
        continue

    valid_Uarx = Uarx[i - 1, np.isfinite(Uarx[i - 1, :])]
    valid_Varx = Varx[i - 1, np.isfinite(Varx[i - 1, :])]

- Условие цикла:

*range(converged + numLastPlots * int(not converged), numLastPlots + 1)*: Этот диапазон определяется условием сходимости. Если процесс сходится (*converged* равно *True*), то цикл будет короче. Это условие управляет тем, сколько последних графиков будет обновлено или сохранено.

- Вычисление индекса:

*l = L + i - numLastPlots*: Рассчитывает актуальный шаг внутри цикла, что используется для меток и файлов вывода.

- Пропуск ненужных итераций:

*if l < L and l in Lplot*: Пропускает итерации, которые не предназначены для вывода на график.

- Фильтрация данных:


*valid_Uarx и valid_Varx*: Выборка только тех значений, которые являются конечными (не *NaN* и не бесконечности), гарантируя, что гистограммы будут корректно построены.

- Визуализация:

In [ ]:
ax1.hist(valid_Uarx, bins=50, density=True, histtype='step', linewidth=1, label=f'L={l}')
    ax1.grid(True)
    ax1.set_ylim([0, np.max(np.histogram(valid_Uarx, bins=50, density=True)[0]) * 3])
    x = (np.histogram(valid_Uarx, bins=50)[1][1:] + np.histogram(valid_Uarx, bins=50)[1][:-1]) / 2

    CNfile.write('\n'.join([f'{l}; {xi:.3f}; {vi}' for xi, vi in zip(x, np.histogram(valid_Uarx, bins=50, density=True)[0])]) + '\n')

    ax2.hist(valid_Varx, bins=50, density=True, histtype='step', linewidth=1, label=f'L={l}')
    ax2.grid(True)
    x = (np.histogram(valid_Varx, bins=50)[1][1:] + np.histogram(valid_Varx, bins=50)[1][:-1]) / 2

    VNfile.write('\n'.join([f'{l}; {xi:.3f}; {vi}' for xi, vi in zip(x, np.histogram(valid_Varx, bins=50, density=True)[0])]) + '\n')
if l in Lplot or l == Lmax or converged:
    xmin = np.min(np.histogram(valid_Varx, bins=50)[1])
    xmax = np.max(np.histogram(valid_Varx, bins=50)[1])
    x = np.linspace(xmin, xmax, 1000)
    absx = np.abs(x)
    valid_Varx = valid_Varx[np.isfinite(valid_Varx)]
    if len(valid_Varx) == 0:
        continuef converged:
    break


- Расчёт статистических моментов:

In [ ]:
b2 = np.sum(valid_Varx ** 2) / len(valid_Varx)
    b4 = np.sum(valid_Varx ** 4) / len(valid_Varx)
    b2 = clamp_values(b2, -1e10, 1e10)
    b4 = clamp_values(b4, -1e10, 1e10)
    a2 = np.sqrt((3 * b2 ** 2 - b4) / 2 + 1e-10)
    a = np.sqrt(a2)
    b = np.sqrt(b2 - a2 + 1e-10)

Вычисление второго (*b2*) и четвёртого (*b4*) моментов распределения.
Применение ограничений к значениям *b2* и *b4* для избежания числовых ошибок.  
Вычисление *a* и *b* для параметризации нормального распределения.

- Построение и отображение графика:

In [ ]:
absVNpdf = norm.pdf(absx, a, b) + norm.pdf(absx, -a, b)
    str_label = f'L={l}, a={a:.3f}, b={b:.3f}, \\neq{np.sqrt(2 * a):.3f}'
    ax2.plot(x, absVNpdf / (1 + np.exp(-x)), label=str_label)
    plt.show()
else:
    ax1.cla()
    ax2.cla()

if converged:
    break

In [ ]:
tend = datetime.now()
print("Duration:", tend - tstart)

CNfile.close()
VNfile.close()

ax1.set_title(f'CN hist, d_c={dc}, d_v={dv}, σ={sigma}')
ax1.legend()

ax2.set_title(f'VN hist, d_c={dc}, d_v={dv}, σ={sigma}')
ax2.legend()

fig.savefig(f'{fname_pref}_L={L}.pdf')


*tend = datetime.now()*: Фиксация времени окончания выполнения.

- Полный код:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from scipy.stats import norm
from sklearn.neighbors import KernelDensity
from scipy.interpolate import interp1d


def kde(data, probability_fcn="cdf", bandwidth=None):
    if bandwidth is None:
        bandwidth = 1.0
    kde = KernelDensity(kernel='gaussian', bandwidth=bandwidth).fit(data[:, None])
    x = np.linspace(min(data), max(data), 1000)
    log_dens = kde.score_samples(x[:, None])
    density = np.exp(log_dens)
    if probability_fcn == "cdf":
        cdf = np.cumsum(density)
        cdf /= cdf[-1]
        return cdf, x, bandwidth
    else:
        return density, x, bandwidth


def randkde(data, dim):
    data = np.nan_to_num(data, nan=0.0, posinf=np.finfo(np.float64).max, neginf=np.finfo(np.float64).min)
    F, x, bw = kde(data, probability_fcn="cdf")
    F[0] = 0
    F[-1] = 1
    inv_cdf = interp1d(F, x, kind='linear', fill_value="extrapolate")
    total_elements = np.prod(dim)
    uniform_random_samples = np.random.rand(total_elements)
    y = inv_cdf(uniform_random_samples).reshape(dim)
    return y, F, x, bw


def filter_infinite_values(data):
    return np.nan_to_num(data, nan=0.0, posinf=np.finfo(np.float64).max, neginf=-np.finfo(np.float64).max)


def clamp_values(data, min_value, max_value):
    return np.clip(data, min_value, max_value)


def safe_arctanh(x):
    epsilon = 1e-10
    x = np.clip(x, -1 + epsilon, 1 - epsilon)
    return np.arctanh(x)


dv = 3
dc = 6
sigma = 0.75

N = int(1e6)
mu = 2 / sigma ** 2
n = dc - 1
m = dv - 1

fname_pref = f'dv={dv}dc={dc}s={sigma}'
CNfile_path = f'{fname_pref}CNhist.csv'
VNfile_path = f'{fname_pref}VNhist.csv'
CNfile = open(CNfile_path, 'w')
VNfile = open(VNfile_path, 'w')
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 12))

Lmax = 30
Lplot = np.arange(1, Lmax + 1)
numLastPlots = 1

VmNn = np.random.normal(mu, np.sqrt(2 * mu), (m, N, n))
VmNn = filter_infinite_values(VmNn)
VmNn = clamp_values(VmNn, -1000, 1000)
Uarx = np.zeros((numLastPlots, N))
Varx = np.zeros((numLastPlots, N))
for i in range(numLastPlots):
    Varx[i, :] = VmNn[0, :, 0]

tstart = datetime.now()
tcurr = datetime.now()

for L in range(1, Lmax + 1):
    if L % 10 == 0:
        print(L, datetime.now() - tcurr, datetime.now() - tstart)
        tcurr = datetime.now()

    UmN = 2 * safe_arctanh(np.prod(np.tanh(VmNn / 2), axis=2))
    UmN = filter_infinite_values(UmN)
    UmN = clamp_values(UmN, -1000, 1000)
    UmNV = np.minimum(UmN, 1000)
    V = np.sum(UmN, axis=0) + np.random.normal(mu, np.sqrt(2 * mu), N)
    V = filter_infinite_values(V)
    V = clamp_values(V, -1000, 1000)

    for i in range(numLastPlots - 1):
        Varx[i, :] = Varx[i + 1, :]
        Uarx[i, :] = Uarx[i + 1, :]
    Varx[numLastPlots - 1, :] = V
    Uarx[numLastPlots - 1, :] = UmN[0, :]

    VmNn = randkde(V, (m, N, n))[0]
    VmNn = filter_infinite_values(VmNn)
    VmNn = clamp_values(VmNn, -1000, 1000)

    converged = np.sum(V < 0) <= 0
    for i in range(converged + numLastPlots * int(not converged), numLastPlots + 1):
        l = L + i - numLastPlots
        if l < L and l in Lplot:
            continue

        valid_Uarx = Uarx[i - 1, np.isfinite(Uarx[i - 1, :])]
        valid_Varx = Varx[i - 1, np.isfinite(Varx[i - 1, :])]

        ax1.hist(valid_Uarx, bins=50, density=True, histtype='step', linewidth=1, label=f'L={l}')
        ax1.grid(True)
        ax1.set_ylim([0, np.max(np.histogram(valid_Uarx, bins=50, density=True)[0]) * 3])
        x = (np.histogram(valid_Uarx, bins=50)[1][1:] + np.histogram(valid_Uarx, bins=50)[1][:-1]) / 2

        CNfile.write('\n'.join(
            [f'{l}; {xi:.3f}; {vi}' for xi, vi in zip(x, np.histogram(valid_Uarx, bins=50, density=True)[0])]) + '\n')

        ax2.hist(valid_Varx, bins=50, density=True, histtype='step', linewidth=1, label=f'L={l}')
        ax2.grid(True)
        x = (np.histogram(valid_Varx, bins=50)[1][1:] + np.histogram(valid_Varx, bins=50)[1][:-1]) / 2

        VNfile.write('\n'.join(
            [f'{l}; {xi:.3f}; {vi}' for xi, vi in zip(x, np.histogram(valid_Varx, bins=50, density=True)[0])]) + '\n')

        if l in Lplot or l == Lmax or converged:
            xmin = np.min(np.histogram(valid_Varx, bins=50)[1])
            xmax = np.max(np.histogram(valid_Varx, bins=50)[1])
            x = np.linspace(xmin, xmax, 1000)
            absx = np.abs(x)
            valid_Varx = valid_Varx[np.isfinite(valid_Varx)]
            if len(valid_Varx) == 0:
                continue
            b2 = np.sum(valid_Varx ** 2) / len(valid_Varx)
            b4 = np.sum(valid_Varx ** 4) / len(valid_Varx)
            b2 = clamp_values(b2, -1e10, 1e10)
            b4 = clamp_values(b4, -1e10, 1e10)
            a2 = np.sqrt((3 * b2 ** 2 - b4) / 2 + 1e-10)
            a = np.sqrt(a2)
            b = np.sqrt(b2 - a2 + 1e-10)
            absVNpdf = norm.pdf(absx, a, b) + norm.pdf(absx, -a, b)
            str_label = f'L={l}, a={a:.3f}, b={b:.3f}, \\neq{np.sqrt(2 * a):.3f}'
            ax2.plot(x, absVNpdf / (1 + np.exp(-x)), label=str_label)
            plt.show()
        else:
            ax1.cla()
            ax2.cla()

    if converged:
        break

tend = datetime.now()
print("Duration:", tend - tstart)

CNfile.close()
VNfile.close()

ax1.set_title(f'CN hist, d_c={dc}, d_v={dv}, σ={sigma}')
ax1.legend()

ax2.set_title(f'VN hist, d_c={dc}, d_v={dv}, σ={sigma}')
ax2.legend()

fig.savefig(f'{fname_pref}_L={L}.pdf')


- Проблемы и решения: 

*Проблемы с переполнением*: Использование ограничений значений и безопасных математических операций.  
*Ошибки деления на ноль*: Введение малых смещений в данные перед выполнением функций, которые не могут обрабатывать крайние значения.